In [6]:
#export
from pytex.core.Precedence import PRECEDENCE
from pytex.LatexInterface.LatexFormatters import LatexBasicFormatter
from pytex.LatexInterface import Operator
from functools import partial

In [7]:
#export
class BasicOperator:
    def __init__(self, precedence, combine):
         self._precedence, self._combine = precedence, combine
    def __call__(self, *args):
        return self._combine(*args)
    @property
    def precedence(self):
        return self._precedence

In [8]:
#export
class InvertibleOperator(BasicOperator):
    def __init__(self, precedence, combine, invert_combine=None):
        super().__init__(precedence, combine)
        self._invert_combine = invert_combine
        
    def __call__(self, *args):
        return super().__call__(*args)
    
    def __invert__(self):
        if self._invert_combine is not None:
            return InvertibleOperator(self.precedence, self._invert_combine, self._combine)
        else:
            return self

def _infix_combine(op):
    return partial(LatexBasicFormatter.binary_operation_output, op)

class InvertibleInfixOperator(InvertibleOperator):
    def __init__(self, precedence, op, invert_op=None):
        c, invc = _infix_combine(op), _infix_combine(invert_op)
        if invert_op is None: 
            super().__init__(precedence, c)
        else: 
            super().__init__(precedence, c, invc)

In [9]:
#export
Plus = InvertibleInfixOperator(PRECEDENCE.ARITHMATIC_ADD, Operator.plus)
Minus = InvertibleInfixOperator(PRECEDENCE.ARITHMATIC_ADD, Operator.minus)
Equal = InvertibleInfixOperator(PRECEDENCE.EQUALITY, Operator.eq, Operator.neq)
LessThan = InvertibleInfixOperator(PRECEDENCE.COMPARE, Operator.lt, Operator.nless)
LessThanEqual = InvertibleInfixOperator(PRECEDENCE.COMPARE, Operator.leq, Operator.nleq)
GreaterThan = InvertibleInfixOperator(PRECEDENCE.COMPARE, Operator.gt, Operator.ngtr)
GreaterThanEqual = InvertibleInfixOperator(PRECEDENCE.COMPARE, Operator.geq, Operator.ngeq)
Space = InvertibleInfixOperator(PRECEDENCE.EQUALITY, Operator.space)

In [10]:
#export
Negate = InvertibleOperator(PRECEDENCE.POWER, 
                            LatexBasicFormatter.negate, 
                            LatexBasicFormatter.unnegate)

In [11]:
(~Negate)('-1')

'1'

In [12]:
#export
mult_combine = _infix_combine(Operator.multiply)
frac_combine = LatexBasicFormatter.fraction

Fraction = InvertibleOperator(PRECEDENCE.ARITHMATIC_MULT, frac_combine, mult_combine)
Multiply = InvertibleOperator(PRECEDENCE.ARITHMATIC_MULT, mult_combine, frac_combine)

In [13]:
#export
power_combine = _infix_combine(Operator.power)
Root = InvertibleOperator(PRECEDENCE.POWER, LatexBasicFormatter.root, power_combine)
Power = InvertibleOperator(PRECEDENCE.POWER, power_combine, LatexBasicFormatter.root)

In [14]:
print ((~Multiply)('1', '2'))

\frac{1}{2}


In [15]:
(~Plus)(1,2), Minus(1,2), Multiply(1,2), Equal(1,2)

('1 + 2', '1 - 2', '1 * 2', '1 = 2')

In [16]:
print((~Root)('a','b'))

a ^ b


In [17]:
e = (~Plus)(1,2)
print(e)

1 + 2


$$
1 \neq 2 \\
\sqrt[\leftroot{-1}\uproot{1}b]{a} \\
a ^ b
$$

In [19]:
! python3 ../notebook2script.py Operator.ipynb

Converted /home/default/Documents/py-latex/pytex/notebooks/core/Operator.ipynb to /home/default/Documents/py-latex/pytex/pytex/core/Operator.py
